### setting

In [2]:
import pandas as pd 
import numpy as np
import matplotlib.pyplot as plt

%matplotlib inline
plt.rcParams['font.family'] = 'AppleGothic'
plt.rcParams['font.size'] = 15
plt.rcParams['figure.figsize'] = (7, 5)
import seaborn as sns

import warnings
warnings.filterwarnings('ignore')

pd.options.display.max_columns=None

import os
os.chdir('/Users/jina/Downloads/h-and-m-personalized-fashion-recommendations')

In [3]:
transaction = pd.read_csv('transactions_train.csv') #학습 데이터(메타데이터로, 다른 데이터를 설명하는 데이터)
article = pd.read_csv('articles.csv') #거래 데이터
customer = pd.read_csv('customers.csv') #고객 데이터
sample = pd.read_csv('sample_submission.csv') #제출 샘플

In [4]:
from datetime import datetime, timedelta

last_date = transaction['t_dat'].max()
last_date = datetime.strptime(last_date, '%Y-%m-%d')

last_week = last_date - timedelta(days=7)
last_month = last_date - timedelta(days=30)
last_3month = last_date - timedelta(days=30*3)

In [5]:
transaction_week = transaction[transaction.t_dat >= str(last_week)]
transaction_month = transaction[transaction.t_dat >= str(last_month)]
transaction_3month = transaction[transaction.t_dat >= str(last_3month)]

### 0 user

#### TOP12(simple)

In [7]:
#// TODO: user count와 기본 count / 0user 인터넷주문?
popular = transaction_month.groupby("article_id").size()
popular_list = list(popular.nlargest(12).index)

#### TOP12(mean,med) 동일

In [8]:
transaction_month_mean = transaction_month[transaction_month['price'] <= transaction_month['price'].mean()]
popular_mean = transaction_month_mean.groupby("article_id").size()
popular_mean_list = list(popular.nlargest(12).index)

#### TOP12(age)

In [9]:
merged_transaction = pd.merge(left = customer , right = transaction_month, how = "right", on = "customer_id")
for i in range(20,80,10):
    cond1 = (merged_transaction['age'] > i-10)
    cond2 = (merged_transaction['age'] < i)
    age_transaction = merged_transaction.loc[cond1 & cond2]
    popular = age_transaction.groupby("article_id").size()
    popular_list = list(popular.nlargest(12).index)
    print(popular_list)        

[715624001, 448509014, 706016001, 685814003, 797988002, 915526001, 918292001, 850917001, 685814001, 552716001, 918522001, 915526002]
[915526001, 918292001, 706016001, 751471001, 898694001, 448509014, 915529003, 909370001, 863583001, 863583002, 714790020, 918522001]
[915529003, 918292001, 768912001, 909370001, 158340001, 933706001, 706016001, 898694001, 751471001, 915529001, 863595006, 751471043]
[751471001, 915529003, 751471043, 706016001, 863595006, 884319001, 865929003, 850917001, 783346001, 673677002, 852584001, 579541001]
[751471043, 751471001, 896152002, 915529003, 751471038, 863595006, 762846031, 924243001, 918522001, 783346001, 893059005, 910601003]
[896152002, 751471043, 933706001, 924243001, 896169002, 579541001, 751471001, 910601003, 918522001, 930380001, 678942001, 863595006]


#### 유입의 / 이달/ 탑12

In [6]:
in_transaction = transaction.drop_duplicates(subset='customer_id', keep='first', inplace=False, ignore_index=False)

In [10]:
transaction_month_in = in_transaction[in_transaction.t_dat >= str(last_month)]

In [14]:
inflow_list_month = transaction_month_in.groupby('article_id')['article_id'].count()
inflow_list_month = inflow_list_month.sort_values(ascending=False)
inflow_list_month.nlargest(12)

article_id
715624001    98
915526001    96
568601006    65
918292001    63
552716001    62
850917001    59
456163060    54
448509014    54
924243001    54
579541001    53
751471001    52
898694001    52
Name: article_id, dtype: int64

#### 유입의 / 이달/ 탑12 (연령별)

In [16]:
merged_transaction = pd.merge(left = customer , right = transaction_month_in, how = "right", on = "customer_id")
for i in range(20,80,10):
    cond1 = (merged_transaction['age'] > i-10)
    cond2 = (merged_transaction['age'] < i)
    age_transaction = merged_transaction.loc[cond1 & cond2]
    popular = age_transaction.groupby("article_id").size()
    popular_list = list(popular.nlargest(12).index)
    print(popular_list)       

[715624001, 552716001, 639448001, 685814003, 448509014, 748355003, 568601006, 915526001, 706016001, 715624010, 456163060, 685814001]
[915526001, 715624001, 758034001, 918292001, 898694001, 874110016, 909370001, 568601006, 924243001, 448509014, 850917001, 856270002]
[775382001, 810169017, 534164032, 810169002, 810170016, 534164001, 534164033, 754792004, 918292001, 562245046, 579541001, 719655001]
[456163086, 456163060, 805000001, 751471001, 794321011, 715624010, 751471038, 889661001, 568601006, 579541001, 610776002, 783346001]
[896152002, 751471001, 796210001, 805000007, 783346001, 874891001, 751471043, 909911001, 910601003, 912100003, 918836001, 568601006]
[579541001, 579541075, 751471043, 874754002, 910601003, 796210001, 904567002, 918836001, 930380001, 568601043, 751471001, 845790002]


### 1-2user

In [113]:
emp2 = pd.DataFrame(transaction_month.groupby(['customer_id','article_id']).size()).reset_index(inplace=False)  
one_user = emp2[emp2.iloc[:,2] == 1]
one_user.head()

,customer_id,article_id,0
0,00000dbacae5abe5e23885899a1fa44253a17956c6d1c3...,568601043,1
1,000058a12d5b43e67d225668fa1f8d618c13dc232df0ca...,794321007,1
2,0000757967448a6cb83efb3ea7a3fb9d418ac7adf2379d...,448509014,1
3,0000757967448a6cb83efb3ea7a3fb9d418ac7adf2379d...,719530003,1
4,00009d946eec3ea54add5ba56d5210ea898def4b46c685...,516859008,1


In [115]:
many_user = emp2[emp2.iloc[:,2] != 1]
many_user.head()

,customer_id,article_id,0
14,0000e9a4db2da4e9c68558ad1e7df570d02767b213ec6b...,880553001,2
17,000172a9c322560c849754ffbdfdb2180d408aa7176b94...,685814001,3
41,0003e867a930d0d6842f923d6ba7c9b77aba33fe2a0fbf...,880017001,2
46,00040239317e877c77ac6e79df42eb2633ad38fcac09fc...,875272011,2
47,00040239317e877c77ac6e79df42eb2633ad38fcac09fc...,875272012,2


In [123]:
many_user[many_user['article_id'] == one_user['article_id'][0]]['customer_id']

406996    68cfad94d2410b92d4c1ad466e4fde1306f3f427e21ac0...
461784    773d1bddfb0c689f040c722a692cbecc92c5bb50b66c98...
560909    906993bf87170a0e9ab24f7aaf0ec473b3e647efa9c8b9...
561126    9075fa0e0a945dd3eb04b417813cd90cfa22a36e49b28b...
763099    c4a43252d519542c91bf85f0e313e91886bd1cdcf32a20...
956442    f6d4b424598dd25c7dd3e2b06b30ffc1c0bb89ac50e3c1...
Name: customer_id, dtype: object

### common user

In [9]:
from gensim import corpora 
from gensim import models
from wordcloud import WordCloud
import matplotlib.pyplot as plt

In [65]:
user2idx = {} # uid_to_idx
for i, l in enumerate(many_user['customer_id'].unique()):
    user2idx[l] = i
    
item2idx = {} # iid_to_idx
for i, l in enumerate(many_user['article_id'].unique()):
    item2idx[l] = i

idx2user = {i: user for user, i in user2idx.items()} #idx_ti_uid
idx2item = {i: item for item, i in item2idx.items()} #idx_to_iid

# many_user['customer_id'] = user2idx(many_user['customer_id'])
# many_user['customer_id'] = [many_user.iloc[i:,0] for i in user2idx(many_user.iloc[i:,0])]

In [62]:
many_user.iloc[0,:]

customer_id    0000e9a4db2da4e9c68558ad1e7df570d02767b213ec6b...
article_id                                             880553001
0                                                              2
Name: 14, dtype: object

### lda 방법

In [21]:
from sklearn.decomposition import LatentDirichletAllocation
from sklearn.datasets import make_multilabel_classification
from scipy.sparse import csr_matrix
import implicit

In [44]:
# many_user.groupby('customer_id')['article_id'].sum()
many_user['article'] = many_user['article_id'] * many_user[0]
many_user

,customer_id,article_id,0,article
14,0000e9a4db2da4e9c68558ad1e7df570d02767b213ec6b...,880553001,2,1761106002
17,000172a9c322560c849754ffbdfdb2180d408aa7176b94...,685814001,3,2057442003
41,0003e867a930d0d6842f923d6ba7c9b77aba33fe2a0fbf...,880017001,2,1760034002
46,00040239317e877c77ac6e79df42eb2633ad38fcac09fc...,875272011,2,1750544022
47,00040239317e877c77ac6e79df42eb2633ad38fcac09fc...,875272012,2,1750544024
...,...,...,...,...
991596,fffcc686584b3388a3afe410e3555e3557f556382b2f60...,840604001,3,2521812003
991597,fffcc686584b3388a3afe410e3555e3557f556382b2f60...,841298001,2,1682596002
991609,fffef3b6b73545df065b521e19f64bf6fe93bfd450ab20...,748269009,2,1496538018
991617,fffef3b6b73545df065b521e19f64bf6fe93bfd450ab20...,881919001,2,1763838002


In [17]:
user2idx = {} # uid_to_idx
for i, l in enumerate(many_user['customer_id'].unique()):
    user2idx[l] = i
    
item2idx = {} # iid_to_idx
for i, l in enumerate(many_user['article_id'].unique()):
    item2idx[l] = i

idx2user = {i: user for user, i in user2idx.items()} #idx_ti_uid
idx2item = {i: item for item, i in item2idx.items()} #idx_to_iid

In [19]:
row, col, dat = many_user['customer_id'].tolist(), many_user['article_id'].tolist(), [1]*len(many_user)
row = [user2idx[r] for r in row]
col = [item2idx[c] for c in col]

In [22]:
train_matrix = csr_matrix((dat, (row,col)), shape=(1 + np.max(row), 1 + np.max(col)))
print(train_matrix.shape)

(57472, 11848)


In [23]:
train_matrix_csr = train_matrix.tocsr()

In [ ]:
k = 50
reg = 0.01
model_imp = implicit.als.AlternatingLeastSquares(factors=k, regularization=reg, iterations=70)
model_imp.fit(train_matrix_csr)

### split

In [3]:
transaction = transaction.sort_values(['t_dat','customer_id'], ascending=True)
transaction

,t_dat,customer_id,article_id,price,sales_channel_id
0,2018-09-20,000058a12d5b43e67d225668fa1f8d618c13dc232df0ca...,663713001,0.050831,2
1,2018-09-20,000058a12d5b43e67d225668fa1f8d618c13dc232df0ca...,541518023,0.030492,2
2,2018-09-20,00007d2de826758b65a93dd24ce629ed66842531df6699...,505221004,0.015237,2
3,2018-09-20,00007d2de826758b65a93dd24ce629ed66842531df6699...,685687003,0.016932,2
4,2018-09-20,00007d2de826758b65a93dd24ce629ed66842531df6699...,685687004,0.016932,2
...,...,...,...,...,...
31788319,2020-09-22,fff2282977442e327b45d8c89afde25617d00124d0f999...,929511001,0.059305,2
31788320,2020-09-22,fff2282977442e327b45d8c89afde25617d00124d0f999...,891322004,0.042356,2
31788321,2020-09-22,fff380805474b287b05cb2a7507b9a013482f7dd0bce0e...,918325001,0.043203,1
31788322,2020-09-22,fff4d3a8b1f3b60af93e78c30a7cb4cf75edaf2590d3e5...,833459002,0.006763,1


In [4]:
valid_mask = transaction['t_dat'] >= '2020-09-16'
train_mask = (transaction['t_dat'] >= '2020-09-09') & (transaction['t_dat'] < '2020-09-16')
valid_transaction = transaction.loc[valid_mask]
train_transaction = transaction.loc[train_mask]
valid_transaction.drop(['t_dat'], axis=1, inplace=True)
train_transaction.drop(['t_dat'], axis=1, inplace=True)

In [5]:
top_12 = list(train_transaction['article_id'].value_counts()[:12].index)
print(top_12)

[909370001, 865799006, 918522001, 924243001, 448509014, 751471001, 809238001, 918292001, 762846027, 809238005, 673677002, 923758001]


### 제출

In [ ]:
transaction_mat = transaction.groupby(['customer_id', 'article_id'])['price'].agg('count')
transaction_mat = transaction_mat.reset_index() 
transaction_mat

In [ ]:
transaction_mat = transaction_mat.assign(
    user_id = transaction_mat['customer_id'].map(user2idx),
    item_id = transaction_mat['article_id'].map(item2idx)
)

transaction_mat = transaction_mat.drop(["customer_id", "article_id"], axis=1)
transaction_mat.info()

useridx = transaction_mat['user_id'].apply(lambda x: idx2user[x]).values
itemidx = transaction_mat['item_id'].apply(lambda x: idx2item[x]).values

In [19]:
submission = pd.read_csv("sample_submission.csv")
submission

In [20]:
customer_to_summit = set(submission['customer_id'])
customer_trained = set(user2idx.keys())

In [22]:
submission_sample = submission.drop("prediction", axis=1)

In [ ]:
popularity_simple_str = " ".join([str(i) for i in top_12])
submission_sample.assign(prediction=popularity_simple_str).to_csv("./submissions/submission_simple_popularity.csv", index=False)